*Packages + directory structure:*

In [1]:
import os, sys, numpy as np, pickle, pandas as pd, math, matplotlib as mpl, matplotlib.pyplot as plt, seaborn, warnings
repo = 'GPM_v05'
directory = {'curr': os.getcwd(),
             'main': os.getcwd().split(repo)[0]+repo}
directory.update({'py': directory['main']+'\\py_main',
                  'work': directory['main']+'\\work_folder',
                  'data': os.getcwd()+'\\Data',
                  'gams': os.getcwd()+'\\gamsmodels'})
os.chdir(directory['py'])
import DataBase, DataBase_wheels, nesting_tree, DB2Gams, excel2py, regex_gms
from DataBase import gpy_symbol
os.chdir(directory['curr'])

In [2]:
try:
    if clean_up is True:
        temp_gamsdb = [filename for filename in os.listdir(directory['work']) if filename.endswith((".gdx",".gms",".lst",".g00",".pf",".opt",".gmy"))]
        for temp in temp_gamsdb:
            try:
                os.remove(directory['work']+'\\'+temp)
            except OSError as err:
                print('The file' + temp + ' is still active and was not deleted.')
except NameError:
    print('No clean-up of work-folder')

No clean-up of work-folder


*Simple routines for running setting up some models:*

In [2]:
def sort_blocks_last(d):
    return {**{k:v for k,v in d.items() if not k.endswith('blocks.gms')}, **d}

In [3]:
def apply_settings(gmodel):
    gmodel.groups = {k:k for k in gmodel.settings.get_conf('g_endo').union(gmodel.settings.get_conf('g_exo'))}
    [gmodel.settings.files.__setitem__(k,v) for k,v in gmodel.settings.functions.items()];
    write_to_db = [k for k,v in gmodel.settings.databases.items() if v == gmodel.database][0]
    with open(gmodel.settings.data_folder+'\\'+gmodel.settings.name+'_sets.gms',"w") as file:
        [file.writelines(c) for c in [gmodel.write_sets(), gmodel.write_aliased_sets(), gmodel.write_sets_other(), gmodel.write_sets_load(write_to_db), 
                                      gmodel.write_parameters(),gmodel.write_parameters_load(write_to_db)]];
    gmodel.settings.files[gmodel.settings.name+'_sets.gms'] = gmodel.settings.data_folder
    gmodel.settings.files.update(gmodel.settings.mfiles)
    with open(gmodel.settings.data_folder+'\\'+gmodel.settings.name+'_groupsload.gms', "w") as file:
        [file.writelines(c) for c in [gmodel.write_groups_load(write_to_db)]];
    gmodel.settings.files[gmodel.settings.name+'_groupsload.gms'] = gmodel.settings.data_folder

*Merge extra attributes:*

In [4]:
def merge_list_of_settings_extra(gsettings,ls):
    gsettings.functions = {k:v for s in ls for k,v in s.functions.items()}
    gsettings.mfiles = sort_blocks_last({k:v for s in ls for k,v in s.mfiles.items()})

*Plottings:*

In [1]:
%matplotlib inline
plt.style.use('seaborn-whitegrid')
mpl.style.use('seaborn')
plt.rcParams['font.family'] = 'Palatino Linotype'
prop_cycle = plt.rcParams["axes.prop_cycle"]
colors = prop_cycle.by_key()["color"]
long_colors = ['#1f78b4','#a6cee3','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']

In [2]:
def one_graph():
    SMALL_SIZE = 16
    MEDIUM_SIZE = 19
    BIGGER_SIZE = 22
    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
def mult_graphs():
    SMALL_SIZE = 13
    MEDIUM_SIZE = 16
    BIGGER_SIZE = 19
    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title    